In [8]:
!pip install textblob
!pip install pydot
!pip install pydotplus
!pip install graphviz
!pip install nltk
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 8.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 KB 8.2 MB/s eta 0:00:000:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [1]:
## libraries
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import numpy as np
import pandas as pd
import nltk
import sklearn

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('sentiwordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
import string
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
import re

## lexicons
from textblob import TextBlob
from nltk.corpus import sentiwordnet as swn

## visualisation
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, trim, first ,when,to_timestamp,udf, from_unixtime,regexp_replace,date_format,substring,StringType

from pyspark.sql.types import ArrayType, IntegerType,DoubleType
import io
import matplotlib.pyplot as plt

# Suppress warning messages
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hduser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [2]:
spark = SparkSession.builder.master("local").config("spark.driver.memory", "15g").appName("CsvReader").getOrCreate()


In [3]:
## read csv 
twitter_data = spark.read.format("csv").load("file:///home/hduser/Desktop/CA2_TweetAnalysis/ProjectTweets.csv")

In [4]:
twitter_data.show(10)

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [5]:
print(twitter_data.columns)

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5']


In [6]:
# Rename columns
twitter_data = twitter_data.selectExpr("_c1 as id", "_c2 as date", "_c3 as flag", "_c4 as user", "_c5 as text")

In [7]:
twitter_data.show(10)

+----------+--------------------+--------+---------------+--------------------+
|        id|                date|    flag|           user|                text|
+----------+--------------------+--------+---------------+--------------------+
|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nope t...|
|1467812025|Mon Apr 06 22:20:...|NO_QUER

In [8]:
twitter_data.count()

1600000

In [9]:
#check tweet and date format
first_row = twitter_data.first()
print(first_row['text'])

print(first_row['date'])

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Mon Apr 06 22:19:45 PDT 2009


In [10]:
# Use regexp_replace to remove PDT from the date_string
twitter_data = twitter_data.withColumn("date", regexp_replace(twitter_data["date"], " PDT", ""))

first_row=twitter_data.first()
print(first_row['text'])

print(first_row['date'])

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Mon Apr 06 22:19:45 2009


In [11]:
#formated date columns as yyyy mm dd hh:mm:ss format
df=twitter_data
df = df.withColumn("new_date", substring(col("date"), 5,25))
# Use the to_timestamp function to convert the date_string to a timestamp
df = df.withColumn("timestamp", to_timestamp(df["new_date"], "MMM dd HH:mm:ss yyyy"))

# Use the date_format function to format the timestamp as desired
df = df.withColumn("formatted_date", date_format(df["timestamp"], "yyyy MM dd HH:mm:ss"))

# Show the DataFrame
df.show()

+----------+--------------------+--------+---------------+--------------------+--------------------+-------------------+-------------------+
|        id|                date|    flag|           user|                text|            new_date|          timestamp|     formatted_date|
+----------+--------------------+--------+---------------+--------------------+--------------------+-------------------+-------------------+
|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|Apr 06 22:19:45 2009|2009-04-06 22:19:45|2009 04 06 22:19:45|
|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|Apr 06 22:19:49 2009|2009-04-06 22:19:49|2009 04 06 22:19:49|
|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|Apr 06 22:19:53 2009|2009-04-06 22:19:53|2009 04 06 22:19:53|
|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|Apr 06 22:19:57 2009|2009-04-06 22:19:57|2009 04 06 22:19:57|
|1467811193|M

In [12]:
df.columns

['id',
 'date',
 'flag',
 'user',
 'text',
 'new_date',
 'timestamp',
 'formatted_date']

In [13]:
df=df[['id',
 'flag',
 'user',
 'text',
 'formatted_date']]
df.show()

+----------+--------+---------------+--------------------+-------------------+
|        id|    flag|           user|                text|     formatted_date|
+----------+--------+---------------+--------------------+-------------------+
|1467810369|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|2009 04 06 22:19:45|
|1467810672|NO_QUERY|  scotthamilton|is upset that he ...|2009 04 06 22:19:49|
|1467810917|NO_QUERY|       mattycus|@Kenichan I dived...|2009 04 06 22:19:53|
|1467811184|NO_QUERY|        ElleCTF|my whole body fee...|2009 04 06 22:19:57|
|1467811193|NO_QUERY|         Karoli|@nationwideclass ...|2009 04 06 22:19:57|
|1467811372|NO_QUERY|       joy_wolf|@Kwesidei not the...|2009 04 06 22:20:00|
|1467811592|NO_QUERY|        mybirch|         Need a hug |2009 04 06 22:20:03|
|1467811594|NO_QUERY|           coZZ|@LOLTrish hey  lo...|2009 04 06 22:20:03|
|1467811795|NO_QUERY|2Hood4Hollywood|@Tatiana_K nope t...|2009 04 06 22:20:05|
|1467812025|NO_QUERY|        mimismo|@twittera que m

## Mongo DB

In [14]:
#!pip install pymongo

In [24]:
##mongodb connection
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["twitter_data"]
collection = db["tweets"]

In [16]:
# clean collection 
#collection.drop()

In [ ]:
#insert data to mongodb 
rows = df.collect()
data_list = [row.asDict() for row in rows]

#insert
for data in data_list:
    collection.insert_one(data)


In [25]:
#read data from mongodb
# Retrieve the data from the MongoDB collection
mongo_data = list(collection.find().limit(5))
mongo_data

[]

In [ ]:
record_count = collection.count_documents({})
record_count


## Spark SQL

In [14]:
## spark sql
df.createOrReplaceTempView("twitter_data")
result = spark.sql("SELECT * FROM twitter_data")
result.show()

+----------+--------+---------------+--------------------+-------------------+
|        id|    flag|           user|                text|     formatted_date|
+----------+--------+---------------+--------------------+-------------------+
|1467810369|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|2009 04 06 22:19:45|
|1467810672|NO_QUERY|  scotthamilton|is upset that he ...|2009 04 06 22:19:49|
|1467810917|NO_QUERY|       mattycus|@Kenichan I dived...|2009 04 06 22:19:53|
|1467811184|NO_QUERY|        ElleCTF|my whole body fee...|2009 04 06 22:19:57|
|1467811193|NO_QUERY|         Karoli|@nationwideclass ...|2009 04 06 22:19:57|
|1467811372|NO_QUERY|       joy_wolf|@Kwesidei not the...|2009 04 06 22:20:00|
|1467811592|NO_QUERY|        mybirch|         Need a hug |2009 04 06 22:20:03|
|1467811594|NO_QUERY|           coZZ|@LOLTrish hey  lo...|2009 04 06 22:20:03|
|1467811795|NO_QUERY|2Hood4Hollywood|@Tatiana_K nope t...|2009 04 06 22:20:05|
|1467812025|NO_QUERY|        mimismo|@twittera que m

In [15]:
result.count()


1600000

## Sentiment Analysis

In [16]:
# cleaning text 
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()




In [17]:
X=df.select("text")
X.show(10)

+--------------------+
|                text|
+--------------------+
|@switchfoot http:...|
|is upset that he ...|
|@Kenichan I dived...|
|my whole body fee...|
|@nationwideclass ...|
|@Kwesidei not the...|
|         Need a hug |
|@LOLTrish hey  lo...|
|@Tatiana_K nope t...|
|@twittera que me ...|
+--------------------+
only showing top 10 rows



### Textblob

In [31]:
# Download NLTK stopwords data
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to tokenize, remove stopwords, and clean punctuation
def preprocess_text(text):
    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Remove punctuation and convert to lowercase
    clean_words = [word.lower() for word in filtered_words if word.isalpha()]

    # Join the cleaned words into a single string
    return " ".join(clean_words)

# Create a user-defined function (UDF) for text preprocessing
preprocess_udf = udf(preprocess_text, StringType())

# Apply the UDF to the DataFrame
processed_df = X.withColumn("text_processed", preprocess_udf(col("text")))

# Perform TextBlob sentiment analysis on the preprocessed text
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

# Create a UDF for sentiment analysis
sentiment_udf = udf(analyze_sentiment, StringType())

# Apply the UDF to the preprocessed DataFrame
sentiment_df = processed_df.withColumn("sentiment_score", sentiment_udf(col("text_processed")))

# Show the DataFrame with sentiment scores
sentiment_df.show(truncate=False)

[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------+
|text                                                                                                                 |text_processed                                                          |sentiment_score     |
+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  |switchfoot http awww bummer shoulda got david carr third day            |0.2                 |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!      |upset ca update facebook 

In [32]:
sentiment_df.select("text_processed","sentiment_score").show(2,truncate=False)

+------------------------------------------------------------------------+---------------+
|text_processed                                                          |sentiment_score|
+------------------------------------------------------------------------+---------------+
|switchfoot http awww bummer shoulda got david carr third day            |0.2            |
|upset ca update facebook texting might cry result school today also blah|0.0            |
+------------------------------------------------------------------------+---------------+
only showing top 2 rows



In [33]:
sentiment_df.select("text","text_processed").show(2,truncate=False)

+-------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|text                                                                                                               |text_processed                                                          |
+-------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|switchfoot http awww bummer shoulda got david carr third day            |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |upset ca update facebook texting might cry result school today also blah|
+--------------------------------------------

### Vader

In [23]:
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 2.9 MB/s eta 0:00:000:00:01


In [34]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql.types import FloatType

analyzer = SentimentIntensityAnalyzer()

# Define a function to apply Vader sentiment analysis
def analyze_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)["compound"]
    return sentiment_score

# Create a user-defined function (UDF) for sentiment analysis
sentiment_udf = udf(analyze_sentiment, FloatType())

# Apply the UDF to the DataFrame
sentiment_df = sentiment_df.withColumn("sentiment_vader_score", sentiment_udf("text_processed"))

# Show the DataFrame with sentiment scores
sentiment_df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------+---------------------+
|text                                                                                                                 |text_processed                                                          |sentiment_score     |sentiment_vader_score|
+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------+---------------------+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  |switchfoot http awww bummer shoulda got david carr third day            |0.2                 |-0.3818              |
|is upset that he can't update his Facebook by texting i

In [29]:
sentiment_df.select("text_processed","sentiment_vader_score").show(2,truncate=False)

+------------------------------------------------------------------------+---------------------+
|text_processed                                                          |sentiment_vader_score|
+------------------------------------------------------------------------+---------------------+
|switchfoot http awww bummer shoulda got david carr third day            |-0.3818              |
|upset ca update facebook texting might cry result school today also blah|-0.7269              |
+------------------------------------------------------------------------+---------------------+
only showing top 2 rows



In [35]:
def get_sentiment_label(score):
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"
    

get_sentiment_label_udf = udf(lambda score: get_sentiment_label(score), StringType())
    
sentiment_df = sentiment_df.withColumn("sentiment", get_sentiment_label_udf(sentiment_df["sentiment_vader_score"]))

# Show the DataFrame with sentiment labels
sentiment_df.show(5,truncate=False)


+-------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+---------------+---------------------+---------+
|text                                                                                                               |text_processed                                                          |sentiment_score|sentiment_vader_score|sentiment|
+-------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+---------------+---------------------+---------+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|switchfoot http awww bummer shoulda got david carr third day            |0.2            |-0.3818              |negative |
|is upset that he can't update his Facebook 

In [36]:
#final df
df_final=df.join(sentiment_df, on="text")
df_final.show(2)

KeyboardInterrupt: 